In [16]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])

[*********************100%***********************]  1 of 1 completed


In [17]:
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()


In [18]:
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3

In [19]:
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")

In [20]:
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])

In [21]:
bins = [-0.43, -0.1, 0, 0.1, 0.43]
group_names = ["Severely Oversold","Oversold", "Neutral","Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
df

,date,price,meanavge,meanvalue,status,price-meanavge,move%,Valuation
299,2011-07-21,13.600000,8.704611,4.895389,1,4.895389,0.219479,Overbought
300,2011-07-22,13.700000,8.762722,4.937278,1,4.937278,0.219799,Overbought
301,2011-07-23,13.700000,8.796167,4.903833,1,4.903833,0.217985,Overbought
302,2011-07-24,14.000000,8.801778,5.198222,1,5.198222,0.227974,Overbought
303,2011-07-25,14.100000,8.823000,5.277000,1,5.277000,0.230205,Overbought
...,...,...,...,...,...,...,...,...
4426,2022-12-05,16974.826172,22346.182676,-5371.356504,0,-5371.356504,-0.136603,Severely Oversold
4427,2022-12-06,17089.503906,22279.310213,-5189.806306,0,-5189.806306,-0.131825,Severely Oversold
4428,2022-12-07,16848.126953,22212.162412,-5364.035459,0,-5364.035459,-0.137327,Severely Oversold
4429,2022-12-08,17233.474609,22149.854450,-4916.379840,0,-4916.379840,-0.124834,Severely Oversold


In [22]:
pricefrommean = df.meanvalue.iloc[-1].round(2)
pricefrommean

-4956.18

In [23]:
currentzone = df.Valuation.iloc[-1]
currentzone

'Severely Oversold'

In [24]:
# Average length of days price stayed under meanaverage or undervalued/severely oversold
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
averageunder


560

In [25]:
# Average length of days from previous cycle peak to reach new all time high
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
averagep2p


1138

In [26]:
# Days since last all time high
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]
sincealltimehigh

396

In [40]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = sincealltimehigh,
    mode = "gauge+number+delta",
    title = {'text': "Average length of days from previous cycle peak to reach new all time high"},
    delta = {'reference': averagep2p},
    gauge = {'axis': {'range': [None, averagep2p]},
    'bar': {'color': "orange"},}))
    
fig.show()

In [38]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0]},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()

In [37]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = sincealltimehigh,
    mode = "gauge+number+delta",
    title = {'text': "Average length of days price has stayed under meanaverage or undervalued/severely oversold"},
      delta = {'reference': averageunder},
    gauge = {'axis': {'range': [None, averageunder],},
    'bar': {'color': "orange"},}))


fig.show()